In [1]:
# For interactive plots, comment the next line
%pylab inline
# For interactive plots, uncomment the next line
# %pylab ipympl
import warnings, os
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


# Cloud Processing
* Obtaining AWS Credentials
* Setting up AWS Credentials
* Creating PODPAC Lambda function
* Evaluating PODPAC Nodes

# Obtaining AWS Credentials
* Create AWS Account
* Log into [console.aws.amazon.com](https://console.aws.amazon.com)
* Services --> IAM --> users 
* This will bring up a table of users
* Click on your username
* Click on the 'Security credentials' tab
* Click on "Create access key"
* Save your AWS_ACCESS_KEY_ID (this can be recovered)
* Save your AWS_SECRET_ACCESS_KEY (this can ONLY be saved at this stage)

<img src="../../images/aws-credentials-create.png" style='margin-left:auto;margin-right:auto;width:80%;'/>

<img src="../../images/aws-credentials-create-id-key.png" style="margin-left:auto;margin-right:auto;width:80%;">

# Setting Up AWS Credentials
We need to provide AWS credentials to PODPAC. This includes:
* AWS_ACCESS_KEY_ID
* AWS_SECRET_ACCESS_KEY

PODPAC can get credentials by:
* Providing credentials when instantiating an AWS Node
* From `podpac.settings`
* From a configured AWS CLI installed for the user

```python
from podpac.managers import aws

# Specify AWS credentials when creating a session
session = aws.Session(aws_access_key_id=AWS_ACCESS_KEY_ID, 
                      aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
```

```python
# Specify AWS credentials in settngs and save
podpac.settings['AWS_ACCESS_KEY_ID'] = AWS_ACCESS_KEY_ID
podpac.settings['AWS_SECRET_ACCESS_KEY'] = AWS_SECRET_ACCESS_KEY
podpac.settings.save()

# Create session
session = aws.Session()  # Keys come from podpac.settings
```

# Creating PODPAC Lambda Function
Now that we have AWS credentials, PODPAC can automatically create the needed resources
* Create an S3 bucket for storing inputs and outputs to AWS Lambda function
* Create the necessary IAM roles so that different resources will have the correct permissions
* Create the AWS Lambda function
* Create an API Gateway endpoint (for WMS requests, for example)
* Configure logging

In [2]:
import podpac
from podpac import settings
from podpac.managers import aws

In [ ]:

# Specify the name for the S3 Bucket
settings["S3_BUCKET_NAME"] = "podpac-test-autogen-10000"  # Must be globally unique (AWS-wide!)

# Create a PODPAC AWS Lambda node
node = aws.Lambda()

In [ ]:
# Create all the AWS resources
# This can take up to 15-20 seconds if all resources need to be created
node.build()
node

# Evaluating PODPAC Nodes
PODPAC Nodes can now be evaluated using AWS Lambda functions:
* Through Python
* By using the API gateway endpoint and OGC-compliant requests
* By uploading files to S3 Buckets (for debugging)

In [14]:
# Get the Pipeline definition for the drought-monitor application (from previous notebook)
pipeline_json = open('pipeline_category.json', 'r').read()
print(pipeline_json)

{
    "DroughtMonitorCategory": {
        "node": "datalib.drought_monitor.DroughtMonitorCategory",
        "attrs": {
            "altkey": "alt",
            "crs": null,
            "datakey": "d0",
            "latkey": "lat",
            "lonkey": "lon",
            "timekey": "time"
        },
        "style": {
            "clim": [
                0,
                0.6
            ]
        },
        "source": "s3://podpac-internal-test/drought_parameters.zarr",
        "interpolation": "nearest"
    },
    "YearSubstituteCoordinates": {
        "node": "core.algorithm.coord_select.YearSubstituteCoordinates",
        "attrs": {
            "substitute_eval_coords": true,
            "year": "2018"
        },
        "inputs": {
            "source": "DroughtMonitorCategory"
        }
    },
    "DroughtMonitorCategory_1": {
        "node": "datalib.drought_monitor.DroughtMonitorCategory",
        "attrs": {
            "altkey": "alt",
            "crs": null,
            "da

# Evaluate PODPAC Node on AWS Through Python

In [15]:
# Create the PODPAC Node from the JSON definition
source = podpac.Node.from_json(pipeline_json)

In [16]:
node = aws.Lambda(
    source=source,        # Any PODPAC Node
    download_result=True  # If False, results will reside on S3
)

NameError: name 'aws' is not defined

In [ ]:
# Create Coordinates for testing node
lat = np.linspace(-10, 10, 5)
lon = np.linspace(-10, 10, 5)
coords = Coordinates([lat, lon], ['lat', 'lon'])

In [ ]:
# Evaluate Node
o = node.eval(coords)

# Plot
o.plot(vmin=0, vmax=1); axis('scaled')
pass

# Evaluate PODPAC Node on AWS Through API Gateway
We will:
* Construct a URL that we can use as a WMS endpoint
* Demonstrate using it through a Leaflet Plugin

In [17]:
# To obtain the API Gateway URL


# Displaying WMS Map

In [ ]:
# Import ipyleaflet library
from ipyleaflet import Map,WMSLayer

In [ ]:
# Create WMS Layer
wms = WMSLayer(
    url=API_URL + '?params={"nodes":' + pipeline_json + '}&',
    layers='%PARAMS%', 
    transparent=True,
    crs='EPSG:3857'
)

In [ ]:
# Create and display map
m = Map(layers=(wms, ), center=(42, -72), zoom=3)
m


# Exercises

* Construct a custom Node
* Evaluate it using AWS Lambda through Python
* Evaluate it using AWS API Gateway